In [0]:

pip install pinecone-client

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install sentence_transformers

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the table embedding model from huggingface models hub
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  original_result = python_builtin_import(name, globals, locals, fromlist, level)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-03f64a74-967e-4d0a-a662-4627a8f83a79/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Out[1]: SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [0]:
import pandas as pd

# Read the CSV file
data = sqlContext.sql('select * from `data`')
df = data.toPandas()

def process_value(value):
    if value == 0:
        return True
    elif value == 1:
        return False
    else:
        return value

# Apply the function to each element in the DataFrame
df = df.applymap(process_value)

tables = []
# Iterate through the DataFrame in chunks of 10 rows
chunk_size = 5
for start in range(0, len(df), chunk_size):
    # Select 10 rows starting from the current index
    chunk = df.iloc[start:start+chunk_size]
    tables.append(chunk)


In [0]:
#Converting the table to string
def _preprocess_tables(tables: list):
    processed = []
    # loop through all tables
    for table in tables:
        # convert the table to csv and
        processed_table = "\n".join([table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed

In [0]:
# format all the dataframes in the tables list
processed_tables = _preprocess_tables(tables)


In [0]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='6315210d-75de-4bdd-b3c1-ffcb5ca3b35c')


In [0]:
# you can choose any name for the index
index_name = "table-qa"

# Check if the table-qa index exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(index_name)

In [0]:

from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(processed_tables), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(processed_tables))
    # extract batch
    batch = processed_tables[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()
    # create unique IDs ranging from zero to the total number of tables in the dataset
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()
     

  0%|          | 0/6 [00:00<?, ?it/s]

Out[7]: {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 349}},
 'total_vector_count': 349}

In [0]:
query = "Give me 5 providers in Tupelo who offer medicaid"
# generate embedding for the query
xq = retriever.encode([query]).tolist()
# query pinecone index to find the table containing answer to the query
result = index.query(vector=xq, top_k=3)
result

Out[25]: {'matches': [{'id': '332', 'score': 0.682276249, 'values': []},
             {'id': '294', 'score': 0.680096567, 'values': []},
             {'id': '188', 'score': 0.661158264, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [0]:
# Get the ids for indices 0, 1, and 2
ids = [int(result["matches"][i]["id"]) for i in range(3)]

# Extract the head (first few rows) of each table corresponding to ids 0, 1, and 2
heads = [tables[id].head() for id in ids]

# Concatenate the heads into a single DataFrame
batch = pd.concat(heads, ignore_index=True)




In [0]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering


tokenizer = AutoTokenizer.from_pretrained("google/tapas-large-finetuned-wtq")
model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-large-finetuned-wtq")
#model.max_seq_length = 8192

In [0]:

batch.fillna('', inplace=True)
batch = batch.sample(frac=1.0, random_state=42, replace=False).reset_index(drop=True).astype(str)
batch


,ID,Provider Name,Provider Address,Provider City,Provider State,PostalCode,Phone1_Type,Phone1_Number,Phone2_Type,Phone2_Number,...,Independent Living,Bridge to Independence,Traumatic Brain Injury / Spinal Cord Injury,Intellectual Disability / Developmental Disability,Private Pay: Credit,Private Pay: Cash,Other,Private Insurance,Medicaid,Private Pay: Check
0,1475,Sam's Club Southaven,465 Goodman Road,Southaven,MS,38671,Main,2283490723,Other,6623493740.0,...,False,False,False,False,False,False,True,False,False,False
1,942,Magnolia Manor of Tupelo,1514 County Road 41,Tupelo,MS,38801,Main,6628426776,,,...,True,True,True,True,False,False,True,True,True,False
2,1661,Tunica Adult Day Care,1530 Edwards Ave.,Tunica,MS,38676,Main,6623632001,,,...,True,True,True,True,True,True,True,True,True,True
3,944,Magnolia Personal Care Home,174 Cliff Mitchell Road,Picayune,MS,39466,Main,6017984007,,,...,True,True,True,True,True,True,True,True,True,True
4,1471,Salvation Army - Tupelo*,527 Carnation Street,Tupelo,MS,38801,Main,6628429222,,,...,False,False,True,True,True,True,True,True,True,True
5,1474,Sam's Club Gulfport,10431 Old Highway 49,Gulfport,MS,39503,Main,2288324441,Other,2288324848.0,...,False,False,False,False,False,False,True,False,False,False
6,1663,Tuxedo Nutrition Site/ Salvation Army Lodge*,610 B Street,Meridian,MS,39301,Main,6014851814,,,...,False,False,False,False,True,True,True,True,True,True
7,1662,Tupelo Nursing and Rehabilitation Center,1901 Briar Ridge Road,Tupelo,MS,38804,Main,6628440675,,,...,True,True,True,True,True,True,True,True,True,True
8,945,Magnolia Place,1515 Munsford Place,New Albany,MS,38652,Main,6625340046,,,...,True,True,True,True,True,True,True,True,True,True
9,1665,U.S.D.A. Rural Development Office/Gulfport,12238 Ashley Drive,Gulfport,MS,39503,Main,2288310881,,,...,False,False,False,False,True,True,True,True,True,True


In [0]:
# Encode the table and query
inputs = tokenizer(table=batch, queries=query, padding="max_length", return_tensors="pt")

# Pass the inputs to the model
outputs = model(**inputs)

# Interpret the logits to get the answer
logits = outputs.logits
logits_agg = outputs.logits_aggregation

# Get the answer from the model's output
predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
    inputs,
    logits.detach(),
    logits_agg.detach()
)

answers = []
for coordinates in predicted_answer_coordinates:
    cell_values = []
    for coord in coordinates:
        try:
            cell_values.append(batch.iat[coord])
        except IndexError:
            print(f"Index {coord} is out of bounds for the DataFrame with shape {batch.shape}")
            continue
    if cell_values:
        answers.append(", ".join(cell_values))

print("Answer:", answers)


Answer: ["Magnolia Manor of Tupelo, Salvation Army  - Tupelo*, Sam's Club Gulfport, Tupelo Nursing and Rehabilitation Center, Sam's Club/Tupelo"]
